In [1]:
#to explore: the step make game lose = score should be lower? like: [-1,-1,0,0,1,0,0,0,0], next step should be 2, otherwise will lose for sure.
#to explore: should it be another model to explore all possible fastest lose situation? Or it is just put some noise will resolve
#to explore: add some noise into input/ output during training
#to explore: remove duplication on training data & score
#resolved - to explore: seperate game engine & cnn into seperate py file
#to explore: what should be a better DQ_ratio?
#to explore: create a chat to show training journey in a pic.
#to explore: Should that be using all pre-training data always everytime train the model??
#to explore: some scenarios AI always loss, may due to no noise on inout
#to explore: if j % 2 == 0:latest version, here needs to be re-visit, may impact final result.
#to explore: 
#to explore: 

import os.path
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch.optim as optim
import numpy as np
from shutil import copyfile

#self created lib in py file format
from Game_Agent import game_agent
from DL_Network_Model import Net
from Funtion_Bank import data_input

torch.set_default_tensor_type('torch.cuda.FloatTensor')

file_name_model_latest_version = 'model_latest_version.pt'
file_name_model_last_version = 'model_last_version.pt'
input_file_path = '../Data/training_data_input.csv'
score_file_path = '../Data/training_data_score.csv'
input_file_path_initiate = '../Data/training_data_input_initiate.csv'
score_file_path_initiate = '../Data/training_data_score_initiate.csv'
training_file_path = '../Data/training_data.csv'

#feel like this part can be improved in a way.
DQ_ratio = 0.80

def training_model():
    
    #1 means there is an existing model saved
    #0 means there is no model saved in folder
    
    if os.path.isfile(file_name_model_latest_version):
        indicator = 1
        input_data = data_input(input_file_path)
        score_data = data_input(score_file_path)
    else:
        indicator = 0
        input_data = data_input(input_file_path_initiate)
        score_data = data_input(score_file_path_initiate)
        #give some initial value is another option
        #input_data = torch.tensor([0,0,0,0,0,0,0,0,0], dtype=torch.float)
        #score_data = torch.tensor([0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11], dtype=torch.float)

    input = input_data
    target = score_data
    # create your optimizer
    net = Net()
    criterion = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr=0.1)
    if indicator == 1:
        state_dict_last_version = torch.load('model_latest_version.pt')['state_dict']
        net.load_state_dict(state_dict_last_version)
        #to explore function to load optimizer from existing model
        #optimizer.load_state_dict(torch.load('model_latest_version.pt')['optimizer'])
    # in your training loop:
    for i in range(1,3000):
        #to explore a batch job for training
        #to explore how to save best performance model during 6000 epochs
        optimizer.zero_grad()   # zero the gradient buffers
        output = net(input)
        loss = criterion(output, target)
        loss.backward()
        if i % 999 == 0:
            print(loss)
        optimizer.step()    # Does the update
    
    if indicator == 1:
        copyfile(file_name_model_latest_version, file_name_model_last_version)
    else:
        torch.save({'state_dict': net.state_dict(),'optimizer': optimizer.state_dict()}, file_name_model_last_version)
    torch.save({'state_dict': net.state_dict(),'optimizer': optimizer.state_dict()}, file_name_model_latest_version)


def main():
    tie = 0
    win = 0
    loss = 0
    
    #load model
    model_last_version = Net()
    state_dict_last_version = torch.load(file_name_model_last_version)['state_dict']
    model_last_version.load_state_dict(state_dict_last_version)
    
    model_latest_version = Net()
    state_dict_latest_version = torch.load(file_name_model_latest_version)['state_dict']
    model_latest_version.load_state_dict(state_dict_latest_version)
    
    training_data = data_input(training_file_path)
    #print(training_data)
    
    #initiallization
    #data_original = [1,0,0,-1,0,0,1,-1,0]
    #data_original = [0,1,0,0,0,0,-1,0,0]
    #data_original = [-1,1,0,1,1,-1,-1,-1,1]
    #data_original = [-1,1,1,1,1,-1,-1,-1,1]
    #data_original = [0,0,0,0,0,0,0,0,0]
    #torch_tensor = torch.tensor(data_original, dtype=torch.float)
    #print('torch_tensor 1st')
    #print(torch_tensor)
    
    for x in range(0,82):
        torch_tensor = training_data[x].clone()
        for j in range(0,9):
            if j == 0:
                ###########another possible scenario: if input without equal occurance of 1 & -1 - pending for coding
                game_a = game_agent(torch_tensor,0)
                if game_a.verify_result() == True:
                    print('not valid input')
                    break
                elif 0 not in torch_tensor:
                    print('input tie & with 9 value')
                    break
                else:
                    #initial
                    input_status = []
                    next_action_taken = []
                    score = []
                    #score = torch.zeros(9)
            torch_tensor_saved = torch_tensor.clone()
            input_status.append(torch_tensor_saved.cpu().numpy())
            #predict
            if j % 2 == 0:
                next_vision = model_latest_version(torch_tensor)
            else:
                next_vision = model_last_version(torch_tensor)

            #print(next_vision)
            current_score = torch.zeros(9)
            #comprehance predict result to make impossible option as '-1'
            for k in range(0,9):
                if torch_tensor[k] != 0:
                    next_vision[k] = -1.1
                    current_score[k] = -1.1
            next_step = next_vision.argmax()
            #save next action

            next_action_taken.append(next_step.cpu().numpy())
            score.append(current_score.cpu().numpy())
            #generate result
            torch_tensor[next_step] = 1
            game_a = game_agent(torch_tensor,0)
            # varify status of result
            if game_a.verify_result() == True:
                #print(game_a.verify_result())
                #calculate DQ
                if j % 2 == 0:
                    win += 1
                else:
                    loss += 1
                #print('finished')
                ############ here as an enhancement on model, it could encourage model for a faster win strategy
                current_final_score = 1
                #update score
                for k in range(0, len(next_action_taken)):
                    score[-(k+1)][next_action_taken[-(k+1)]] = ((-1 * DQ_ratio)**(k))

                #####################break for what?
                break
            else:
                if 0 not in torch_tensor:
                    #print('tie')
                    tie = tie + 1
                    break
                else:
                    #generate next input
                    torch_tensor = torch_tensor * -1
        input_status_df = pd.DataFrame(input_status)
        input_status_df.to_csv('../Data/training_data_input.csv', index=False, mode='a', header=False)
        next_action_taken_df = pd.DataFrame(next_action_taken)
        next_action_taken_df.to_csv('../Data/training_data_next_action_taken.csv', index=False, mode='a', header=False)
        score_df = pd.DataFrame(score)
        score_df.to_csv('../Data/training_data_score.csv', index=False, mode='a', header=False)
    print("tie" + str(tie))
    print("win" + str(win))
    print("loss" + str(loss))
    

    
if __name__ == "__main__":
    button = 1
    if button == 1:
        for loops in range(0,5):
            training_model()
            main()
    else:
        main()

tensor(0.0295, grad_fn=<MseLossBackward>)
tensor(0.0294, grad_fn=<MseLossBackward>)
tensor(0.0294, grad_fn=<MseLossBackward>)
tie11
win60
loss11
tensor(0.0292, grad_fn=<MseLossBackward>)
tensor(0.0292, grad_fn=<MseLossBackward>)
tensor(0.0291, grad_fn=<MseLossBackward>)
tie9
win59
loss14
tensor(0.0292, grad_fn=<MseLossBackward>)
tensor(0.0292, grad_fn=<MseLossBackward>)
tensor(0.0290, grad_fn=<MseLossBackward>)
tie9
win59
loss14
tensor(0.0289, grad_fn=<MseLossBackward>)
tensor(0.0288, grad_fn=<MseLossBackward>)
tensor(0.0288, grad_fn=<MseLossBackward>)
tie9
win60
loss13
tensor(0.0287, grad_fn=<MseLossBackward>)
tensor(0.0286, grad_fn=<MseLossBackward>)
tensor(0.0284, grad_fn=<MseLossBackward>)
tie12
win60
loss10


In [2]:
#to explore: a UI for game - it can be a excel or something.
#defect: win, it shows loss as there is no goto


import os.path
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch.optim as optim
import numpy as np
from shutil import copyfile

#self created lib in py file format
from Game_Agent import game_agent
from DL_Network_Model import Net
from Funtion_Bank import data_input


torch.set_default_tensor_type('torch.cuda.FloatTensor')

file_name_model_latest_version = 'model_latest_version.pt'
file_name_model_last_version = 'model_last_version.pt'
input_file_path = '../Data/training_data_input.csv'
score_file_path = '../Data/training_data_score.csv'
input_file_path_initiate = '../Data/training_data_input_initiate.csv'
score_file_path_initiate = '../Data/training_data_score_initiate.csv'
training_file_path = '../Data/training_data.csv'

#feel like this part can be improved in a way.
DQ_ratio = 0.80

def main():
    tie = 0
    win = 0
    loss = 0
    
    #load model
    model_last_version = Net()
    state_dict_last_version = torch.load(file_name_model_last_version)['state_dict']
    model_last_version.load_state_dict(state_dict_last_version)
    
    model_latest_version = Net()
    state_dict_latest_version = torch.load(file_name_model_latest_version)['state_dict']
    model_latest_version.load_state_dict(state_dict_latest_version)
    
    training_data = data_input(training_file_path)
    #print(training_data)
    
    #initiallization
    #data_original = [1,0,0,-1,0,0,1,-1,0]
    #data_original = [0,1,0,0,0,0,-1,0,0]
    #data_original = [-1,1,0,1,1,-1,-1,-1,1]
    #data_original = [-1,1,1,1,1,-1,-1,-1,1]
    data_original = [-1,-1,0,0,1,0,0,0,0]
    torch_tensor = torch.tensor(data_original, dtype=torch.float)
    #print('torch_tensor 1st')
    #print(torch_tensor)
    

    ###########another possible scenario: if input without equal occurance of 1 & -1 - pending for coding
    game_a = game_agent(torch_tensor,0)
    if game_a.verify_result() == True:
        print('not valid input')
    elif 0 not in torch_tensor:
        print('input tie & with 9 value')
    else:
        #initial
        input_status = []
        next_action_taken = []
        score = []
        #score = torch.zeros(9)
    torch_tensor_saved = torch_tensor.clone()
    input_status.append(torch_tensor_saved.cpu().numpy())
    #predict

    next_vision = model_latest_version(torch_tensor)

    print('next_vision')
    print(next_vision)
    current_score = torch.zeros(9)
    #comprehance predict result to make impossible option as '-1'
    for k in range(0,9):
        if torch_tensor[k] != 0:
            next_vision[k] = -1.1
            current_score[k] = -1.1
    next_step = next_vision.argmax()
    print('next_step')
    print(next_step)
    #save next action

    next_action_taken.append(next_step.cpu().numpy())
    score.append(current_score.cpu().numpy())
    #generate result
    torch_tensor[next_step] = 1
    game_a = game_agent(torch_tensor,0)
    # varify status of result
    if game_a.verify_result() == True:
        #print(game_a.verify_result())
        #calculate DQ
        win += 1
        #print('finished')
        ############ here as an enhancement on model, it could encourage model for a faster win strategy
        current_final_score = 1
        #update score
        for k in range(0, len(next_action_taken)):
            score[-(k+1)][next_action_taken[-(k+1)]] = ((-1 * DQ_ratio)**(k))
    else:
        if 0 not in torch_tensor:
            #print('tie')
            tie = tie + 1
        else:
            #generate next input
            torch_tensor = torch_tensor * -1

    print("tie" + str(tie))
    print("win" + str(win))
    print("loss" + str(loss))
    
    
    
if __name__ == "__main__":
    button = 0
    if button == 1:
        for loops in range(0,5):
            training_model()
            main()
    else:
        main()

next_vision
tensor([-1.0229, -1.0290, -0.0322, -0.0310, -1.1401, -0.0284,  0.0648,  0.0527,
        -0.0791], grad_fn=<AddBackward0>)
next_step
tensor(6)
tie0
win0
loss0


In [6]:
# an option to let AI do first
import os.path
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch.optim as optim
import numpy as np
from shutil import copyfile



#self created lib in py file format
from Game_Agent import game_agent
from DL_Network_Model import Net
from Funtion_Bank import data_input


from tkinter import *

file_name_model_latest_version = 'model_latest_version.pt'
file_name_model_last_version = 'model_last_version.pt'

DQ_ratio = 0.80

tie = 0
win = 0
loss = 0

#load model
model_last_version = Net()
state_dict_last_version = torch.load(file_name_model_last_version)['state_dict']
model_last_version.load_state_dict(state_dict_last_version)

model_latest_version = Net()
state_dict_latest_version = torch.load(file_name_model_latest_version)['state_dict']
model_latest_version.load_state_dict(state_dict_latest_version)


def trigger_AI(data_original):
    game_result = "Ongoing.."
    #print(training_data)
    
    #initiallization
    #data_original = [1,0,0,-1,0,0,1,-1,0]
    #data_original = [0,1,0,0,0,0,-1,0,0]
    #data_original = [-1,1,0,1,1,-1,-1,-1,1]
    #data_original = [-1,1,1,1,1,-1,-1,-1,1]
    torch_tensor = torch.tensor(data_original, dtype=torch.float)
    #print('torch_tensor 1st')
    #print(torch_tensor)
    
    ###########another possible scenario: if input without equal occurance of 1 & -1 - pending for coding
    game_a = game_agent(torch_tensor,0)
    if game_a.verify_result() == True:
        print('Win')
        game_result = "Win"
    elif 0 not in torch_tensor:
        print('input tie & with 9 value')
        game_result = "input tie & with 9 value"
    else:
        #initial
        input_status = []
        next_action_taken = []
        score = []
        #score = torch.zeros(9)
    #torch_tensor_saved = torch_tensor.clone()
    #input_status.append(torch_tensor_saved.cpu().numpy())
    #predict

    next_vision = model_latest_version(torch_tensor)

    print('next_vision')
    print(next_vision)
    current_score = torch.zeros(9)
    #comprehance predict result to make impossible option as '-1'
    for k in range(0,9):
        if torch_tensor[k] != 0:
            next_vision[k] = -1.1
            current_score[k] = -1.1
    next_step = next_vision.argmax()
    print('next_step')
    print(next_step)
    #save next action

    #next_action_taken.append(next_step.cpu().numpy())
    #score.append(current_score.cpu().numpy())
    #generate result
    torch_tensor[next_step] = 1
    game_a = game_agent(torch_tensor,0)
    # varify status of result
    if game_a.verify_result() == True:
        #print(game_a.verify_result())
        #calculate DQ
        #win += 1
        #print('finished')
        game_result = 'Loss'
        ############ here as an enhancement on model, it could encourage model for a faster win strategy
        #update score
    else:
        if 0 not in torch_tensor:
            #print('tie')
            game_result = 'tie'
            #tie = tie + 1
        else:
            #generate next input
            torch_tensor = torch_tensor * -1
    return next_step, game_result







window = Tk()

window.title("Welcome to Game made by HX")
window.geometry('400x400')
lbl = Label(window, text="Hello")


data_original = [0,0,0,0,0,0,0,0,0]
game_board = ['','','','','','','','','']

#lbl.grid(column=0, row=0)
#txt = Entry(window,width=10)
#txt.grid(column=1, row=0)
btn_00_text = StringVar()
btn_01_text = StringVar()
btn_02_text = StringVar()
btn_10_text = StringVar()
btn_11_text = StringVar()
btn_12_text = StringVar()
btn_20_text = StringVar()
btn_21_text = StringVar()
btn_22_text = StringVar()
#


def refresh_game_board(data_original):
    for i in range(0,9):
        if data_original[i] == -1:
            game_board[i] = 'o'
        elif data_original[i] == 1:
            game_board[i] = 'x'
        else:
            game_board[i] = ''
    btn_00_text.set(game_board[0])
    btn_01_text.set(game_board[1])
    btn_02_text.set(game_board[2])
    btn_10_text.set(game_board[3])
    btn_11_text.set(game_board[4])
    btn_12_text.set(game_board[5])
    btn_20_text.set(game_board[6])
    btn_21_text.set(game_board[7])
    btn_22_text.set(game_board[8])
    
    

def button_00_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_00_text.get() == '':
        btn_00_text.set("o")
        data_original[0] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        data_original[next_step] = 1
        #print(data_original)
        refresh_game_board(data_original)
        print(game_board)
        lbl.configure(text= game_result)
        
def button_01_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_01_text.get() == '':
        btn_01_text.set("o")
        data_original[1] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        data_original[next_step] = 1
        #print(data_original)
        refresh_game_board(data_original)
        print(game_board)
        lbl.configure(text= game_result)


        
def button_02_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_02_text.get() == '':
        btn_02_text.set("o")
        data_original[2] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        data_original[next_step] = 1
        #print(data_original)
        refresh_game_board(data_original)
        print(game_board)
        lbl.configure(text= game_result)

def button_10_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_10_text.get() == '':
        btn_10_text.set("o")
        data_original[3] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        data_original[next_step] = 1
        #print(data_original)
        refresh_game_board(data_original)
        print(game_board)
        lbl.configure(text= game_result)


def button_11_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_11_text.get() == '':
        btn_11_text.set("o")
        data_original[4] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        data_original[next_step] = 1
        #print(data_original)
        refresh_game_board(data_original)
        print(game_board)
        lbl.configure(text= game_result)

def button_12_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_12_text.get() == '':
        btn_12_text.set("o")
        data_original[5] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        data_original[next_step] = 1
        #print(data_original)
        refresh_game_board(data_original)
        print(game_board)
        lbl.configure(text= game_result)

def button_20_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_20_text.get() == '':
        btn_20_text.set("o")
        data_original[6] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        data_original[next_step] = 1
        #print(data_original)
        refresh_game_board(data_original)
        print(game_board)
        lbl.configure(text= game_result)



def button_21_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_21_text.get() == '':
        btn_21_text.set("o")
        data_original[7] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        data_original[next_step] = 1
        #print(data_original)
        refresh_game_board(data_original)
        print(game_board)
        lbl.configure(text= game_result)
        
        
def button_22_clicked():
    #lbl.configure(text="Button was clicked !!")
    if btn_22_text.get() == '':
        btn_22_text.set("o")
        data_original[8] = -1
        print(data_original)
        next_step, game_result = trigger_AI(data_original)
        print(next_step)
        data_original[next_step] = 1
        #print(data_original)
        refresh_game_board(data_original)
        print(game_board)
        lbl.configure(text= game_result)
        
        
        
        
        
        
        
        
        
        


btn_00 = Button(window, textvariable=btn_00_text, command=button_00_clicked, height = 2, width = 4)
btn_00.grid(row=0, column=0)
btn_00_text.set('')

btn_01 = Button(window, textvariable=btn_01_text, command=button_01_clicked, height = 2, width = 4)
btn_01.grid(row=0, column=1)

btn_02 = Button(window, textvariable=btn_02_text, command=button_02_clicked, height = 2, width = 4)
btn_02.grid(row=0, column=2)

btn_10 = Button(window, textvariable=btn_10_text, command=button_10_clicked, height = 2, width = 4)
btn_10.grid(row=1, column=0)

btn_11 = Button(window, textvariable=btn_11_text, command=button_11_clicked, height = 2, width = 4)
btn_11.grid(row=1, column=1)

btn_12 = Button(window, textvariable=btn_12_text, command=button_12_clicked, height = 2, width = 4)
btn_12.grid(row=1, column=2)

btn_20 = Button(window, textvariable=btn_20_text, command=button_20_clicked, height = 2, width = 4)
btn_20.grid(row=2, column=0)

btn_21 = Button(window, textvariable=btn_21_text, command=button_21_clicked, height = 2, width = 4)
btn_21.grid(row=2, column=1)

btn_22 = Button(window, textvariable=btn_22_text, command=button_22_clicked, height = 2, width = 4)
btn_22.grid(row=2, column=2)

lbl = Label(window, text="Hello")
lbl.grid(column=4, row=4)



window.mainloop()

[0, 0, -1, 0, 0, 0, 0, 0, 0]
next_vision
tensor([-0.0175, -0.0164, -1.1056, -0.0888,  0.0410, -0.0543,  0.0258, -0.0842,
        -0.0204], grad_fn=<AddBackward0>)
next_step
tensor(4)
tensor(4)
['', '', 'o', '', 'x', '', '', '', '']
[0, 0, -1, 0, 1, -1, 0, 0, 0]
next_vision
tensor([-0.0441,  0.0910, -1.0853, -0.0851, -1.0351, -1.0311, -0.0680, -0.0020,
         0.2151], grad_fn=<AddBackward0>)
next_step
tensor(8)
tensor(8)
['', '', 'o', '', 'x', 'o', '', '', 'x']
[-1, 0, -1, 0, 1, -1, 0, 0, 1]
next_vision
tensor([-1.0503,  0.0504, -1.1625, -0.2457, -1.0078, -1.1254,  0.0182, -0.1435,
        -1.0510], grad_fn=<AddBackward0>)
next_step
tensor(1)
tensor(1)
['o', 'x', 'o', '', 'x', 'o', '', '', 'x']
[-1, 1, -1, 0, 1, -1, 0, -1, 1]
next_vision
tensor([-0.9203, -1.0883, -1.2302, -0.1177, -1.0750, -1.1266, -0.1854, -0.9711,
        -1.0950], grad_fn=<AddBackward0>)
next_step
tensor(3)
tensor(3)
['o', 'x', 'o', 'x', 'x', 'o', '', 'o', 'x']
[-1, 1, -1, 1, 1, -1, -1, -1, 1]
input tie & with 9 va

In [21]:
import tkinter as tk

root = tk.Tk()

def update_btn_text():
    btn_text.set("b")

btn_text = tk.StringVar()
btn = tk.Button(root, textvariable=btn_text, command=update_btn_text)
btn_text.set("a")


root.mainloop()